In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 23.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.8 MB/s eta 0:00:00:00:0100:01
  Attempting u

In [3]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig,AutoTokenizer

In [4]:
print(torch.cuda.is_available())
#print("bitsandbytes 版本:", bitsandbytes.__version__)

True


NameError: name 'bitsandbytes' is not defined

In [5]:
device = "cuda:0"
# 定义量化配置
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16)

In [6]:
# 加载模型和分词器
model_q = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-1.3b",
    quantization_config=quantization_config,
    device_map=device
)
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-1.3b",
    device_map=device
)
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

# 测试生成
inputs = tokenizer("Once upon a time", return_tensors="pt").to(model.device)
outputs_q = model_q.generate(**inputs, max_new_tokens=50)
print("model_q output:",tokenizer.decode(outputs_q[0], skip_special_tokens=True))
outputs = model.generate(**inputs, max_new_tokens=50)
print("model output:",tokenizer.decode(outputs[0], skip_special_tokens=True))

inputs = tokenizer("从前有座山", return_tensors="pt").to(model.device)
outputs_q = model_q.generate(**inputs, max_new_tokens=50)
print("model_q output:",tokenizer.decode(outputs_q[0], skip_special_tokens=True))
outputs = model.generate(**inputs, max_new_tokens=50)
print("model output:",tokenizer.decode(outputs[0], skip_special_tokens=True))

del model_q  # GPU内存不足，删除不再使用的模型
del model
torch.cuda.empty_cache()

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

2025-07-24 04:32:26.466622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753331546.826899      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753331546.930775      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

model_q output: Once upon a time, there was a man who was a very good friend of mine.
He was a very good friend of mine.
He was a very good friend of mine.
He was a very good friend of mine.
He was a very good
model output: Once upon a time, there was a man who was a very good friend of mine.
He was a very good friend of mine, and he was a very good friend of mine.
He was a very good friend of mine, and he was a very good
model_q output: 从前有座山路沙江路沙江路沙江路沙江路沙江路�
model output: 从前有座山路港河河港港港港港港港港港港港港�


In [7]:
# 加载模型和分词器
model_name="EleutherAI/gpt-neo-1.3B"
model_q = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map=device
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 测试生成
inputs = tokenizer("Once upon a time", return_tensors="pt").to(model.device)
outputs_q = model_q.generate(**inputs, max_new_tokens=50)
print("model_q output:",tokenizer.decode(outputs_q[0], skip_special_tokens=True))
outputs = model.generate(**inputs, max_new_tokens=50)
print("model output:",tokenizer.decode(outputs[0], skip_special_tokens=True))

inputs = tokenizer("从前有座山", return_tensors="pt").to(model.device)
outputs_q = model_q.generate(**inputs, max_new_tokens=50)
print("model_q output:",tokenizer.decode(outputs_q[0], skip_special_tokens=True))
outputs = model.generate(**inputs, max_new_tokens=50)
print("model output:",tokenizer.decode(outputs[0], skip_special_tokens=True))

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model_q output: Once upon a time, there was a young man named John who was a good friend of a young woman named Mary. John was a good friend of Mary because he was a good friend of Mary because he was a good friend of Mary because he was a good friend of


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


model output: Once upon a time, I was a young girl, and I was a very good girl. I was a very good girl, and I was a very good girl, and I was a very good girl, and I was a very good girl, and I was a


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


model_q output: 从前有座山西山西山西山西山西山西山西山西山�
model output: 从前有座山路，没有人把它放在山路上，没有人把它放在�


In [11]:
device = "cuda:1"  # cuda:0已内存不足
# 加载模型和分词器
model_name="bigscience/bloom-1b7"
model_q = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map=device
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 测试生成
inputs = tokenizer("Once upon a time", return_tensors="pt").to(model.device)
outputs_q = model_q.generate(**inputs, max_new_tokens=50)
print("model_q output:",tokenizer.decode(outputs_q[0], skip_special_tokens=True))
outputs = model.generate(**inputs, max_new_tokens=50)
print("model output:",tokenizer.decode(outputs[0], skip_special_tokens=True))

inputs = tokenizer("从前有座山", return_tensors="pt").to(model.device)
outputs_q = model_q.generate(**inputs, max_new_tokens=50)
print("model_q output:",tokenizer.decode(outputs_q[0], skip_special_tokens=True))
outputs = model.generate(**inputs, max_new_tokens=50)
print("model output:",tokenizer.decode(outputs[0], skip_special_tokens=True))

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model_q output: Once upon a time, I was a little girl.
I was a little girl.
I was a little girl.
I was a little girl.
I was a little girl.
I was a little girl.
I was a little girl.
I was a little girl.
I
model output: Once upon a time, there was a man who was a great hunter.
He was a great hunter.
He was a great hunter.
He was a great hunter.
He was a great hunter.
He was a great hunter.
He was a great hunter.
He was a
model_q output: 从前有座山,山上有座庙,庙里住着一位老和尚,老和尚的徒弟们都想拜他为师,老和尚却说:"你们都想拜我为师,我却不收你们为徒。" 徒弟们不解
model output: 从前有座山,山上有座庙,庙里有个和尚,和尚每天念经,念经念累了,就坐在庙里休息。 有一天,和尚念经念累了,就坐在庙里休息。 突然,庙里来了一个和尚
